In [3]:
from ctu_crs import CRS97
import numpy as np
from numpy.typing import NDArray
import time
import PoseComposer as pc
import puzzle


In [4]:
robot = CRS97()


Firmware version : MARS8 v1.0 build Pi Oct  2 2017 11:06:45


In [6]:
robot.initialize(home=True)

Exception: Check ready: arm power is off.

In [7]:
robot.initialize(home=False)

In [8]:
maze = puzzle.PuzzleA()
maze.get_reverse_trajectory()

NameError: name 'R' is not defined

In [9]:
direction = [0, 1, 0]
points = maze.get_forward_trajectory()

for i in range(len(points-1)):
    normal = points[i] - points[i+1]
    normal = pc.normalize(normal)
    pose = pc.make_se3_matrix(points[i], direction, normal)
    ik_sols = robot.ik(pose)
    q0 = robot.get_q()
    assert len(ik_sols) > 0
    closest_solution = min(ik_sols, key=lambda q: np.linalg.norm(q - q0))
    feasible = robot.in_limits(closest_solution)
    print(feasible)

True
True
True
True
True
True
True
True
False


IndexError: index 10 is out of bounds for axis 0 with size 10

In [27]:
def check_one_path(maze, hoop_dir):
    points = maze.get_forward_trajectory()
    last_normal = []
    feasibility = True
    for i in range(len(points)):
        if i + 1 < len(points):
            normal = points[i] - points[i+1]
        else:
            normal = last_normal
        normal = pc.normalize(normal)
        pose = pc.make_se3_matrix(points[i], hoop_dir, normal)
        ik_sols = robot.ik(pose)
        q0 = robot.get_q()
        assert len(ik_sols) > 0
        closest_solution = min(ik_sols, key=lambda q: np.linalg.norm(q - q0))
        feasible = robot.in_limits(closest_solution)
        last_normal = normal
        if not feasible:
            feasibility = False        
    return feasibility

In [28]:
points = maze.get_forward_trajectory()
point0 = points[0]
normal0 = point0 - points[1]
positions, vectors = pc.find_options_around_point(point0, normal0)
for vector in vectors:
    feasibility = check_one_path(maze, vector)
    print(f"atempt is feasible: {feasibility}")
        
        

atempt is feasible: False
atempt is feasible: False
atempt is feasible: False
atempt is feasible: False
atempt is feasible: False
atempt is feasible: False
atempt is feasible: False
atempt is feasible: False


In [18]:
positions

[array([0.2 , 0.  , 0.14]),
 array([ 0.2       , -0.09899495,  0.09899495]),
 array([ 2.00000000e-01, -1.40000000e-01,  3.10862447e-17]),
 array([ 0.2       , -0.09899495, -0.09899495]),
 array([ 2.00000000e-01, -3.88578059e-17, -1.40000000e-01]),
 array([ 0.2       ,  0.09899495, -0.09899495]),
 array([ 2.00000000e-01,  1.40000000e-01, -6.21724894e-17]),
 array([0.2       , 0.09899495, 0.09899495])]

In [19]:
vectors

[array([0., 0., 1.]),
 array([ 0.        , -0.70710678,  0.70710678]),
 array([ 0.00000000e+00, -1.00000000e+00,  2.22044605e-16]),
 array([ 0.        , -0.70710678, -0.70710678]),
 array([ 0.00000000e+00, -2.77555756e-16, -1.00000000e+00]),
 array([ 0.        ,  0.70710678, -0.70710678]),
 array([ 0.0000000e+00,  1.0000000e+00, -4.4408921e-16]),
 array([0.        , 0.70710678, 0.70710678])]

In [14]:
def chose_feasible_path(maze):
    points = maze.get_forward_trajectory()
    point0 = points[0]
    normal0 = point0 - points[1]
    options = pc.find_options_around_point(point0, normal0)